In [2]:
import hyperparams

# Copied code from keen-source

Find a cleaner fix later

In [10]:
import os
os.environ["CUBLAS_WORKSPACE_CONFIG"] = ":4096:8"

import torch
import copy

import pandas as pd

import torch.nn as nn
import torch.optim as optim

import numpy as np

import random
from scipy.stats import pearsonr

# Deterministic Run
random_seed = 42
random.seed(random_seed)
np.random.seed(random_seed)
torch.manual_seed(random_seed)
torch.cuda.manual_seed(random_seed)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(random_seed)
torch.set_num_threads(1)
torch.use_deterministic_algorithms(True)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
torch.backends.cudnn.enabled = False


optimizers = {
    "adam": optim.AdamW,
    "sgd": optim.SGD
}


class MLPRegressor(nn.Module):

    def __init__(self, input_size, optimizer, learning_rate, max_iter, T_max=None):
        super(MLPRegressor, self).__init__()
        self.max_iter = max_iter
        self.criterion = nn.MSELoss()
        self.last_activation = torch.sigmoid
        self.layers =  nn.ModuleList(
            [nn.Linear(input_size, 1, bias=False)]
        )
        self.best_weights = []
        self.init_weights = []
        self._initialize_weights()
        self.optimizer = optimizers[optimizer](self.parameters(), lr=learning_rate)
        self.best_train_loss, self.best_test_pearson_corr = 1, -1
        self.initial_train, self.initial_test, self.final_train, self.final_test = None, None, None, None 

    def _initialize_weights(self):
        for layer in self.layers:
            torch.nn.init.kaiming_uniform_(layer.weight, nonlinearity="relu")
            self.init_weights.append(layer.weight)

        for layer in self.layers:
            self.best_weights.append(layer.weight)

    def set_to_best_weights(self):
        for i, weight in enumerate(self.best_weights):
            self.layers[i].weight = weight

    def forward(self, x): 
        x = self.layers[0](x)
        return self.last_activation(x) 

    def predict(self, x):
        with torch.no_grad():
            return self.forward(x)
    
    def validate(self, X_test, y_test):
            preds = self.predict(X_test)
            test_loss = self.criterion(preds, y_test).item()

            result_df = pd.DataFrame(
                {
                    "preds": preds.squeeze(dim=-1).detach().cpu().numpy(), 
                    "target": y_test.squeeze(dim=-1).detach().cpu().numpy()
                }
            )
            test_pearson_corr, test_pearson_p_value = pearsonr(result_df["preds"], result_df["target"])
            return result_df, test_loss, test_pearson_corr, test_pearson_p_value

    def fit(self, X_train, y_train, X_test, y_test):     
        X_test = torch.tensor(X_test.tolist(), dtype=torch.float32).cuda()  
        y_test = torch.tensor(y_test.tolist(), dtype=torch.float32).unsqueeze(dim=1).cuda()
        
        for epoch in range(self.max_iter):
            print(f"Epoch {epoch+1}/{self.max_iter}")
            epoch_train_loss = 0.0
            n_examples = 0
            for batch in X_train:
                # Train
                self.optimizer.zero_grad()
                preds = self.forward(batch[0].to(torch.float32).cuda())
                train_loss = self.criterion(preds, batch[1].to(torch.float32).unsqueeze(dim=1).cuda())
                train_loss.backward()
                self.optimizer.step()
                epoch_train_loss += train_loss.item()
                n_examples += batch[0].shape[0]
            
            # Test 
            epoch_train_loss = epoch_train_loss / y_train.shape[0]
            result_df, test_loss, test_pearson_corr, test_pearson_p_value = self.validate(X_test, y_test)
            
            if epoch == 0:
                self.initial_test = result_df
            if test_pearson_corr > self.best_test_pearson_corr:
                self.best_test_pearson_corr = test_pearson_corr
                self.best_weights = [copy.deepcopy(l.weight)for l in self.layers]
                self.final_test = result_df

            print({"test_loss": test_loss, "train_loss": epoch_train_loss, "test_pearson_corr": test_pearson_corr})     


# Load vectors

In [4]:
import json

import numpy as np
from sklearn.model_selection import train_test_split
from torch.utils.data import TensorDataset, DataLoader

In [5]:
INPUT_NPZ_PATH = 'data/filtered_vectors.npz' 

try:
    with np.load(INPUT_NPZ_PATH) as data:
        norm_reps = data['representations_normalized']
        row_indices = data['row_indices']
        
except FileNotFoundError:
    print(f"Error: The file '{INPUT_NPZ_PATH}' was not found.")
except Exception as e:
    print(f"An unexpected error occurred: {e}")

In [6]:
# Load the JSON data
with open('data/score_by_entity.json', encoding='utf-8') as file:
    data = json.load(file)

# Create a dictionary mapping vector_index to score for O(1) lookup
score_dict = {item['vector_index']: item['score'] for item in data}

# Create scores list in the order determined by row_indices
scores = np.array([score_dict[idx] for idx in row_indices])

In [7]:
X, y = norm_reps, scores

# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=random_seed)

# Convert to PyTorch tensors and create DataLoader
train_dataset = TensorDataset(
    torch.tensor(X_train, dtype=torch.float32),
    torch.tensor(y_train, dtype=torch.float32)
)

# Create DataLoader with batch size
train_loader = DataLoader(train_dataset, batch_size=hyperparams.BATCH_SIZE, shuffle=True)

# Probe model

In [11]:
# Instantiate MLPRegressor with the hyperparameters
model = MLPRegressor(
    input_size=X.shape[1],
    optimizer=hyperparams.OPTIM,
    learning_rate=hyperparams.LR,
    max_iter=hyperparams.MAX_ITER
)
model.cuda()

MLPRegressor(
  (criterion): MSELoss()
  (layers): ModuleList(
    (0): Linear(in_features=3072, out_features=1, bias=False)
  )
)

In [ ]:
model.fit(train_loader, y_train, X_test, y_test)

Epoch 1/20972


C:\Users\sofia\AppData\Local\Temp\ipykernel_23224\1979267127.py:85: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  test_pearson_corr, test_pearson_p_value = pearsonr(result_df["preds"], result_df["target"])


{'test_loss': 0.03678825497627258, 'train_loss': 0.0011479860366662672, 'test_pearson_corr': np.float32(nan)}
Epoch 2/20972
{'test_loss': 0.03678825497627258, 'train_loss': 0.0011073691190873452, 'test_pearson_corr': np.float32(0.0011790118)}
Epoch 3/20972
{'test_loss': 0.03678825497627258, 'train_loss': 0.001105576884605257, 'test_pearson_corr': np.float32(0.008325754)}
Epoch 4/20972
{'test_loss': 0.03678825497627258, 'train_loss': 0.0011049671735838, 'test_pearson_corr': np.float32(0.007995705)}
Epoch 5/20972
{'test_loss': 0.03678825497627258, 'train_loss': 0.001104767539610617, 'test_pearson_corr': np.float32(0.008222784)}
Epoch 6/20972
{'test_loss': 0.03678825497627258, 'train_loss': 0.001105273840775444, 'test_pearson_corr': np.float32(0.008445206)}
Epoch 7/20972
{'test_loss': 0.03678825497627258, 'train_loss': 0.001105280389907182, 'test_pearson_corr': np.float32(0.0086628925)}
Epoch 8/20972
{'test_loss': 0.03678825497627258, 'train_loss': 0.001109763798164672, 'test_pearson_corr